## Production and Transportation Scheduling with (Deep) Reinforcement Learning

### Problem Description.

- Given a set of orders form customers design a plan that
    - selects the appropriate warehouse from which to send the goods
    - selects the right carrier to send them with

Assumption:

- no Warehouse maximal capacity.

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Code Implementation

### Reading in Data and Graphs

In [64]:
plant_ports = pd.read_csv("PlantPorts.csv", index_col=0)
order_list = pd.read_csv("OrderList.csv", index_col=0)
products_plants = pd.read_csv("ProductsPerPlant.csv", index_col=0)
vmi_plants = pd.read_csv("VmiCustomers.csv", index_col=0)
freight_rates = pd.read_csv("FreightRates.csv", index_col=0)
wh_cost = pd.read_csv("WhCosts.csv", index_col=0)
order_list.columns = [i.replace(" ", "_") for i in order_list.columns]
products_plants.columns = [i.replace(" ", "_") for i in products_plants.columns]
plant_ports.columns = [i.replace(" ", "_") for i in plant_ports.columns]
vmi_plants.columns = [i.replace(" ", "_") for i in vmi_plants.columns]

In [65]:
plant_ports_graph = nx.from_pandas_edgelist(plant_ports, source="Plant_Code", target="Port")

### Preparing the Order Table

In [66]:
order_new = order_list.drop(columns=["Order_Date", "Origin_Port", "Carrier", "Plant_Code", "TPT", "Service_Level", "Ship_ahead_day_count", "Ship_Late_Day_count"])
order_new.set_index("Order_ID", inplace=True)
order_new.to_csv("order_new.csv")

### Preparing Freight Table

In [67]:
freight_rates.drop(columns=["dest_port_cd", "Carrier type", "svc_cd"], inplace=True)
freight_rates.to_csv("FreightRates_mod.csv")

### Problem Restrictions

In [68]:
# given a product id, return the plants that can produce this product.
def product_restriction(index):
    data = order_new.loc[index]
    product_id = data["Product_ID"]
    possible_plants = products_plants.loc[products_plants["Product_ID"] == product_id]
    return np.array(possible_plants["Plant_Code"])

In [52]:
# check if a given customer HAS to be serviced by a specific facility, else return all facilities as possibilities.
def customer_restriction(index):
    data = order_new.loc[index]
    Customer_id = data["Customer"]
    possible_plants = vmi_plants.loc[vmi_plants["Customers"] == Customer_id]
    if list(possible_plants["Plant_Code"]) == []:
        return plant_ports["Plant_Code"].unique()
    else:
        return np.array(possible_plants["Plant_Code"])

In [56]:
# combine both the product and vmi restriction. There will be orders for which only one facility can fullfil it.
def check_order(Order_Id, length=True):
    if length:
        return len(np.intersect1d(customer_restriction(Order_Id), product_restriction(Order_Id)))
    else:
        return np.intersect1d(customer_restriction(Order_Id), product_restriction(Order_Id))

In [70]:
# under the restrictions above, we can calculate the number of facilities that can process a given order.
order_new["decision_space_size"] = np.array(list(map(check_order, order_new.index)))

In [ ]:
order_new["decision_space_size"].value_counts()

We can see that for most orders, there is only one facility that can handle the order. For ~1,000 there is no possible facility that can handle the order. 

In [30]:
# given a facility, return the possible ports from which the goods can be sent.
def port_restriction(plant_code):
    return plant_ports.loc[plant_ports.Plant_Code == plant_code, "Port"]

In [40]:
# given the port from which the good is supposed to be shipped, return the possible carriers that can ship it.
def carrier_restriction(orig_port_cd):
    return freight_rates.loc[freight_rates.orig_port_cd == orig_port_cd]

In [ ]:
carrier_restriction("PORT08")

In [33]:
port_restriction("PLANT03")

3    PORT04
Name: Port, dtype: object

### Scheduling orders with Decision Space Size = 1

For these orders, we do not need to worry about assigning orders to the right warehouse.

In [71]:
order_one = order_new.loc[order_new.decision_space_size == 1]

In [74]:
order_one["decision_space"] = np.array(list(map(lambda x: check_order(x, length=False), order_one.index)))

C:\Users\lbmm\AppData\Local\Temp/ipykernel_1756/1167662604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_one["decision_space"] = np.array(list(map(lambda x: check_order(x, length=False), order_one.index)))


In [61]:
order_one["wh_cost"] = order_one["Unit_quantity"]

,Customer,Product_ID,Destination_Port,Unit_quantity,Weight,decision_space_size,decision_space
Order_ID,,,,,,,
1.447296e+09,V55555_53,1700106,PORT09,808,14.300000,1,PLANT16
1.447158e+09,V55555_53,1700106,PORT09,3188,87.940000,1,PLANT16
1.447139e+09,V55555_53,1700106,PORT09,2331,61.200000,1,PLANT16
1.447364e+09,V55555_53,1700106,PORT09,847,16.160000,1,PLANT16
1.447364e+09,V55555_53,1700106,PORT09,2163,52.340000,1,PLANT16
...,...,...,...,...,...,...,...
1.447372e+09,V55555555555555_8,1690628,PORT09,247,0.415441,1,PLANT02
1.447372e+09,V55555555555555_8,1690628,PORT09,352,1.038603,1,PLANT02
1.447328e+09,V55555555555555_8,1690628,PORT09,343,0.692402,1,PLANT02


In [82]:
order_one["wh_cost"] = order_one["decision_space"].apply(lambda x: float(wh_cost.loc[wh_cost.WH == x, "Cost/unit"]))

C:\Users\lbmm\AppData\Local\Temp/ipykernel_1756/449883748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_one["wh_cost"] = order_one["decision_space"].apply(lambda x: float(wh_cost.loc[wh_cost.WH == x, "Cost/unit"]))


In [83]:
order_one.head(2)

,Customer,Product_ID,Destination_Port,Unit_quantity,Weight,decision_space_size,decision_space,wh_cost
Order_ID,,,,,,,,
1.447296e+09,V55555_53,1700106,PORT09,808,14.30,1,PLANT16,1.919808
1.447158e+09,V55555_53,1700106,PORT09,3188,87.94,1,PLANT16,1.919808


### Mapping the Supply Chain as a Bipartite Graph

#### Non Interactive with Networkx

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.set_facecolor("Grey")

# specify layout for the graph
# layout = nx.bipartite_layout(plant_ports_graph, plant_ports["Plant Code"])

layout = nx.bipartite_layout(plant_ports_graph, plant_ports["Plant Code"])

for i in layout:
    if i.startswith("PLANT"):
        layout[i][0] -= 0.1
    else:
        layout[i][0] += 0.1

# we want to map the degree of the node to a color/size
degrees = dict(plant_ports_graph.degree)
maps = [v*100 for v in degrees.values()]

# specify the color map
cmap = plt.cm.Blues

# keyword args that are the same for both functions
kwargs = {"pos":layout, "ax":ax}

nx.draw_networkx_nodes(plant_ports_graph, node_size=maps, node_color=maps, cmap=cmap, **kwargs)
nx.draw_networkx_edges(plant_ports_graph, **kwargs)
nx.draw_networkx_labels(plant_ports_graph, pos=layout)
plt.show()

#### Interactive Version with Plotly.

- code adopted from https://plotly.com/python/network-graphs/

In [9]:
# get starting and ending points of the edges and add them to the graph
layout = nx.bipartite_layout(plant_ports_graph, plant_ports["Plant Code"])

for i in layout:
    if i.startswith("PLANT"):
        layout[i][0] += 0.3
    else:
        layout[i][0] -= 0.3

edge_x = []
edge_y = []

for edge in plant_ports_graph.edges():
    x0, y0 = layout[edge[0]]
    x1, y1 = layout[edge[1]]
    
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#25488e'),
    hoverinfo='none',
    mode='lines')

In [10]:
# get coordinated of nodes and add them to the graph

node_x = []
node_y = []
for node in plant_ports_graph.nodes():
    x, y = layout[node]
    node_x.append(x)
    node_y.append(y)
    
maps = [v for v in degrees.values()]

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=maps,
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

# node_trace.text = ["Number of Links: " + str(i) for i in maps]
node_trace.text = [i + " Number of Links: " + str(degrees[i]) for i in degrees]

In [11]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Supply Chain',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[dict(text="Factories",
                     showarrow=False,
                     xref="paper", yref="paper",
                     x=0.005, y=-0.002 ),
                             dict(text="Ports",
                     showarrow=False,
                     xref="paper", yref="paper",
                     x=0.95, y=-0.002 )],
                xaxis=dict(showgrid=True, zeroline=False, showticklabels=True),
                yaxis=dict(showgrid=True, zeroline=False, showticklabels=True))
                )
fig

In [1]:
(1,2,3) + (1,2)

(1, 2, 3, 1, 2)

In [9]:
def get(elements):
    to_return = 0
    for value in reversed(elements):
        if value % 2 != 0:
            print(True)
            to_return = value
    return to_return
    
get([7,3,4,5])

True
True
True


7

In [18]:
def sum_numbers(start=1, end=10):
    result = 0
    for i in range(start, end):
        result += i
    return result

a = 10
b = 10

sum_numbers(0,4)

6

Most facilities have only one connection to a port. Port 4 is potentially the most important one as it has the most connections to the warehouses.